# TensorRTへの変換

Pytorchで学習したモデルをTensorRTモデルに変換します。  
``road_following/train_model.ipynb``ノートブックの指示に従って、すでに``best_steering_model_xy.pth``を作成していることを想定します。

最初にtorchvisionで提供されている未学習のResNet18モデルを読み込みます。(自前学習した値でモデルを初期化するため、ImageNetで学習済みのモデルである必要がない)  
次に、ResNet18モデル構造の全結合層(fully connected layer)を入れ替えて、JetBotで欲しい出力x,yの2種類を得られるモデル構造にします。  
学習済みモデル``best_steering_model_xy.pth``から値を読み込むことで、ResNet18モデル構造に対して学習した値が適用されます。

In [ ]:
import torchvision
import torch

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load('../best_steering_model_xy.pth'))

TensorRTはGPUでのみ動作可能なため、GPUを利用するようにモデルを設定します。

In [ ]:
device = torch.device('cuda')
model = model.to(device)
model = model.cuda().eval().half()

TorchからTensorRTに変換します。

In [ ]:
from torch2trt import torch2trt 
data = torch.randn((1, 3, 224, 224)).cuda().half()
model_trt = torch2trt(model, [data], fp16_mode=True)

変換が正しく行われたか、変換前と変換後の比較をおこないます。

In [ ]:
output_trt = model_trt(data)
output = model(data)

print(output.flatten()[0:10])
print(output_trt.flatten()[0:10])
print('max error: %f' % float(torch.max(torch.abs(output - output_trt))))

TensorRTモデルとして保存します。

In [ ]:
torch.save(model_trt.state_dict(), 'best_steering_model_xy_trt.pth')

## Next(次)

``live_demo_trt.ipynb``でTensorRTに変換した学習モデルで自動走行します。  
ノートブックメニューから`Kernel`->`Restert Kernel`を選んでJupyter kernelを再起動するか、JetBotを一度再起動してから次に進むとスムーズに進行できます。

[live_demo_trt.ipynb](./live_demo_trt.ipynb) をクリックし、移動します。